# EnergyPlus and TRNSYS Example

In [1]:
import os
from zerobnl import CoSim

You can safely ignore the following error (it will also be in the nodes logs):

RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

-> [Numpy documentation](https://github.com/numpy/numpy/pull/432)

In [2]:
sim = CoSim()

# There are 4 environments in this example.

sim.create_meta_model("MetaPLANT", [("Treturn", "Cdeg"),("mdot_return", "kgs"),("COGw_Flag","-")], [("Tsupply", "Cdeg"),("MDOTsupply", "kgs")]) # Set, Get
sim.create_environment("EnvPLANT", "wrapper_plant.py", "Dockerfile_na")

sim.create_meta_model("MetaMIXER", [("COG_TS", "Cdeg"),("COG_mdot", "kgs")], [("TS", "Cdeg")]) # Set, Get
sim.create_environment("EnvMIXER", "wrapper_mixer.py", "Dockerfile_mixer")

sim.create_meta_model("MetaCDEM", [("Ts_second", "Cdeg")], [("CDEM_TR", "Cdeg"),("CDEM_mdotR", "kgs"),("CDEM_cmf","-")]) # Set, Get
sim.create_environment("EnvCDEM", "wrapper_cdem.py", "Dockerfile_cdem")

sim.create_meta_model("MetaCTRL", [("demandOK", "-")], [("COGwflag", "-")]) # Set, Get
sim.create_environment("EnvCTRL", "wrapper_ctrl.py", "Dockerfile_ctrl")

# There is one model per each environment in this example. 
# OBS!! The name Base# indicates a model. Each model is contained in a node. There can be more nodes in each environment.
sim.add_node("Base0", "MetaPLANT", "EnvPLANT", init_values={"Treturn":40,"mdot_return":0.04,"COGw_Flag":1}, files=["COGplant.fmu"],local=True)
sim.add_node("Base1", "MetaMIXER", "EnvMIXER", init_values={},local=False)
sim.add_node("Base2", "MetaCDEM", "EnvCDEM", init_values={},local=False)
sim.add_node("Base3", "MetaCTRL", "EnvCTRL", init_values={},local=False)

# Links among nodes
sim.add_link("Base0", "Tsupply", "Base1", "COG_TS") # get from Base0 and set to Base1
sim.add_link("Base0", "MDOTsupply", "Base1", "COG_mdot") # get from Base0 and set to Base1
sim.add_link("Base1", "TS", "Base2", "Ts_second") # get from Base1 and set to Base0
sim.add_link("Base2", "CDEM_TR", "Base0", "Treturn") # get from Base1 and set to Base0
sim.add_link("Base2", "CDEM_mdotR", "Base0", "mdot_return") # get from Base1 and set to Base0
sim.add_link("Base2", "CDEM_cmf", "Base3", "demandOK") # get from Base1 and set to Base0
sim.add_link("Base3", "COGwflag", "Base0", "COGw_Flag") # get from Base1 and set to Base0

# Create groups from the simulation sequence. Nodes in the same group run in parallel. 
# A group is defined within the first level of square brackets.
sim.create_sequence([["Base0"], ["Base1"],["Base2"],["Base3"]])
sim.set_time_unit("seconds")
sim.create_steps([3600] * 24)

Once the next step has been launched, logging `INFO :: Waiting for local nodes to run..`, you need to run tho following command `wrapper_eplus.py Base1 GRP1` in the indicated folder (in a dedicated environment) in order to run the local node.

In [3]:
sim.run()

INFO :: Waiting for local nodes to run...
INFO :: Simulation finished in 0 min and 15 sec


If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,IN,Base1,COG_mdot
1,IN,Base0,mdot_return
2,IN,Base0,COGw_Flag
3,IN,Base2,Ts_second
4,OUT,Base2,CDEM_TR
5,OUT,Base1,TS
6,OUT,Base0,Tsupply
7,IN,Base3,demandOK
8,OUT,Base0,MDOTsupply
9,OUT,Base2,CDEM_mdotR


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
for key, value in sim.get_results_by_pattern("OUT*Base0*").items():
    print(key)
    print(value)   

INFO :: Matching results: Base0 - Tsupply
INFO :: Matching results: Base0 - MDOTsupply


OUT||Base0||MDOTsupply
2000-01-01 01:00:00    0.04
2000-01-01 02:00:00    0.04
2000-01-01 03:00:00    0.04
2000-01-01 04:00:00    0.04
2000-01-01 05:00:00    0.04
2000-01-01 06:00:00    0.04
2000-01-01 07:00:00    0.04
2000-01-01 08:00:00    0.04
2000-01-01 09:00:00    0.04
2000-01-01 10:00:00    0.04
2000-01-01 11:00:00    0.04
2000-01-01 12:00:00    0.04
2000-01-01 13:00:00    0.04
2000-01-01 14:00:00    0.04
2000-01-01 15:00:00    0.04
2000-01-01 16:00:00    0.04
2000-01-01 17:00:00    0.04
2000-01-01 18:00:00    0.04
2000-01-01 19:00:00    0.04
2000-01-01 20:00:00    0.04
2000-01-01 21:00:00    0.04
2000-01-01 22:00:00    0.04
2000-01-01 23:00:00    0.04
2000-01-02 00:00:00    0.04
dtype: float64
OUT||Base0||Tsupply
2000-01-01 01:00:00    43.970250
2000-01-01 02:00:00    43.953579
2000-01-01 03:00:00    43.958750
2000-01-01 04:00:00    43.955033
2000-01-01 05:00:00    43.956122
2000-01-01 06:00:00    43.955863
2000-01-01 07:00:00    43.955919
2000-01-01 08:00:00    43.955908
2000-0

In [6]:
import matplotlib.pyplot as plt
for key, value in sim.get_results_by_pattern("OUT*Base1*").items():
    plt.figure(figsize=(18, 8))
    plt.plot(value, "o-", label="Treturn_DH", alpha=1)
    plt.legend()

INFO :: Matching results: Base1 - TS


In [7]:
for key, value in sim.get_results_by_pattern("IN*Base0*").items():
    print(key)
    print(value)

INFO :: Matching results: Base0 - mdot_return
INFO :: Matching results: Base0 - COGw_Flag
INFO :: Matching results: Base0 - Treturn


IN||Base0||COGw_Flag
2000-01-01 01:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 09:00:00    1.0
                      ... 
2000-01-01 17:00:00    1.0
2000-01-01 17:00:00    1.0
2000-01-01 17:00:00    1.0
2000-01-01 18:00:00    1.0
2000-01-01 18:00:00    1.0
2000-01

In [8]:
for key, value in sim.get_results_by_pattern("IN*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - COG_mdot
INFO :: Matching results: Base1 - COG_TS


IN||Base1||COG_TS
2000-01-01 00:00:00    43.970250
2000-01-01 01:00:00    43.970250
2000-01-01 01:00:00    43.970250
2000-01-01 01:00:00    43.970250
2000-01-01 01:00:00    43.953579
2000-01-01 02:00:00    43.953579
2000-01-01 02:00:00    43.953579
2000-01-01 02:00:00    43.953579
2000-01-01 02:00:00    43.958750
2000-01-01 03:00:00    43.958750
2000-01-01 03:00:00    43.958750
2000-01-01 03:00:00    43.958750
2000-01-01 03:00:00    43.955033
2000-01-01 04:00:00    43.955033
2000-01-01 04:00:00    43.955033
2000-01-01 04:00:00    43.955033
2000-01-01 04:00:00    43.956122
2000-01-01 05:00:00    43.956122
2000-01-01 05:00:00    43.956122
2000-01-01 05:00:00    43.956122
2000-01-01 05:00:00    43.955863
2000-01-01 06:00:00    43.955863
2000-01-01 06:00:00    43.955863
2000-01-01 06:00:00    43.955863
2000-01-01 06:00:00    43.955919
2000-01-01 07:00:00    43.955919
2000-01-01 07:00:00    43.955919
2000-01-01 07:00:00    43.955919
2000-01-01 07:00:00    43.955908
2000-01-01 08:00:00    43

In [9]:
for key, value in sim.get_results_by_pattern("OUT*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - TS


OUT||Base1||TS
2000-01-01 01:00:00    43.970250
2000-01-01 02:00:00    43.953579
2000-01-01 03:00:00    43.958750
2000-01-01 04:00:00    43.955033
2000-01-01 05:00:00    43.956122
2000-01-01 06:00:00    43.955863
2000-01-01 07:00:00    43.955919
2000-01-01 08:00:00    43.955908
2000-01-01 09:00:00    43.955910
2000-01-01 10:00:00    43.955909
2000-01-01 11:00:00    43.955910
2000-01-01 12:00:00    43.955910
2000-01-01 13:00:00    43.955910
2000-01-01 14:00:00    43.955910
2000-01-01 15:00:00    43.955910
2000-01-01 16:00:00    43.955910
2000-01-01 17:00:00    43.955910
2000-01-01 18:00:00    43.955910
2000-01-01 19:00:00    43.955910
2000-01-01 20:00:00    43.955910
2000-01-01 21:00:00    43.955910
2000-01-01 22:00:00    43.955910
2000-01-01 23:00:00    43.955910
2000-01-02 00:00:00    43.955910
dtype: float64
